In [35]:
import torch
import torch.nn as nn

In [36]:
GPT_CONFIG_124M = {
    "vocab_size": 50247,  # 词汇表大小
    "context_length": 1024,  # 上下文长度
    "emb_dim": 768,  # 嵌入维度
    "n_heads": 12,  # 注意力头数
    "n_layers": 12, #层数
    "drop_rate": 0.1,  # Dropout率
    "qkv_bias": False  # Query-Key-Value偏置
}

In [37]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out nust be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask", torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill(mask_bool, -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [38]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return (
            0.5
            * x
            * (
                1
                + torch.tanh(
                    torch.sqrt(torch.tensor(2.0 / torch.pi))
                    * (x + 0.044715 * torch.pow(x, 3))
                )
            )
        )

In [39]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [40]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [41]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg: GPT_CONFIG_124M):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"],
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        # 层归一化
        x = self.norm1(x)
        # 多头注意力
        x = self.att(x)
        # 丢弃部分样本
        x = self.drop_shortcut(x)
        # 残差连接
        x = x + shortcut
        # 对前网络模块的残差连接
        shortcut = x
        # 层归一化
        x = self.norm2(x)
        # 前馈神经网络
        x = self.ff(x)
        # 丢弃部分样本
        x = self.drop_shortcut(x)
        # 残差连接
        x = x + shortcut
        return x

In [42]:
torch.manual_seed(123)
# 创建形状为[batch_size,num_tokens,emb_dim]的样本输入
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Ouput shape:", output.shape)

tensor([[[0.2961, 0.5166, 0.2517,  ..., 0.9541, 0.8567, 0.4604],
         [0.2238, 0.3047, 0.3019,  ..., 0.5465, 0.4532, 0.7598],
         [0.6945, 0.2478, 0.4111,  ..., 0.8838, 0.4898, 0.5963],
         [0.0890, 0.7804, 0.9223,  ..., 0.4507, 0.6357, 0.5833]],

        [[0.5716, 0.9297, 0.3396,  ..., 0.0477, 0.4564, 0.2797],
         [0.0936, 0.2211, 0.3806,  ..., 0.3948, 0.4545, 0.4536],
         [0.6788, 0.1741, 0.2084,  ..., 0.5557, 0.5930, 0.0959],
         [0.3894, 0.4083, 0.0662,  ..., 0.9861, 0.9341, 0.1319]]])
tensor([[[ 0.2358,  0.2692,  0.2002,  ...,  0.7233,  0.7167,  0.4196],
         [-0.0754,  0.3154,  0.3019,  ...,  0.2511,  0.4333,  0.7598],
         [ 0.4972,  0.2478,  0.2709,  ...,  0.6593,  0.2648,  0.6607],
         [ 0.0305,  0.7333,  0.9206,  ...,  0.2248,  0.6357,  0.5946]],

        [[ 0.7952,  0.8253,  0.3354,  ..., -0.0049,  0.4283,  0.0294],
         [ 0.0936,  0.1299,  0.4209,  ...,  0.1922,  0.6086,  0.2750],
         [ 0.8270,  0.0773,  0.1001,  ...,  0.43

In [52]:
class GPTModel(nn.Module):
    def __init__(self, cfg: GPT_CONFIG_124M):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        device = in_idx.device
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [54]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Evert effort moves you"
txt2 = "Evert day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
# 编码输入文本
batch = torch.stack(batch, dim=0)
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)

tensor([[[ 0.0000,  0.1472, -1.6157,  ..., -0.9615,  2.0212,  0.8369],
         [-2.4706,  1.5290,  0.3996,  ...,  2.2005, -0.7835, -2.7575],
         [-1.5051, -1.9831,  0.3828,  ..., -1.7232, -1.1892,  1.7786],
         [-0.5718,  0.3978,  1.0057,  ...,  0.6945, -1.9185,  2.3094],
         [ 2.5645, -0.9048, -2.0827,  ..., -2.9734,  0.3255,  0.5302]],

        [[ 0.7828,  0.1472, -1.6157,  ..., -0.9615,  2.0212,  0.8369],
         [-2.4706,  1.5290,  0.3996,  ...,  2.2005, -0.7835, -2.7575],
         [-0.0000, -0.4168,  0.0000,  ..., -2.1209, -1.9936,  0.0000],
         [ 0.7179,  0.0300,  1.0561,  ...,  0.7306, -0.0000,  0.8715],
         [ 4.4317, -1.1251,  0.7268,  ..., -0.9140, -1.3264, -0.1936]]],
       grad_fn=<MulBackward0>)
tensor([[[ 0.3889,  0.0673, -1.7572,  ..., -1.2293,  2.0907,  0.8369],
         [-2.1856,  1.5290,  0.2646,  ...,  1.9384, -0.6679, -2.9005],
         [-1.3062, -2.0158,  0.3481,  ..., -1.8282, -0.9403,  1.5846],
         [-0.2804,  0.3523,  1.0057,  ...,

In [57]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters:{total_params:,}")
print("Token embedding layer shape:",model.tok_emb.weight.shape)
print("Output layer shape:",model.out_head.weight.shape)

Total number of parameters:162,994,176
Token embedding layer shape: torch.Size([50247, 768])
Output layer shape: torch.Size([50247, 768])


In [59]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(
    f"Number of trainable parameters "
    f"considering weight tying: {total_params_gpt2:,}"
)
# 计算总大小(假设float32,每个参数4字节)
total_size_bytes = total_params * 4
# 转换为兆字节
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Number of trainable parameters considering weight tying: 124,404,480
Total size of the model: 621.77 MB


In [81]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # 预测单词的模块
    # idx是当前上下文中的(batch,n_tokens)索引数组
    for _ in range(max_new_tokens):
        # 每次生成一个单词后,重新将其加入序列中
        # 如果当前上下文长度超过模型支持的最大上下文长度则截取
        # 例如,如果LLM只支持5个token,而上下文长度为10
        # 那么只使用最后5个token作为上下文
        idx_cond = idx[:, -context_size:]
        print(idx_cond)
        # 如果idx的长度超过模型支持的上下文长度size,只保留最后size个token
        # 避免溢出
        # 获取预测结果
        with torch.no_grad():  # 在推理阶段,不需要计算梯度,因为没有反向传播
            # 这样可以检查存储开销
            logits = model(idx_cond)
            print(logits.shape)
            # 模型输出结果
        # 只关注最后一个时间步的输出
        # (batch,n_tokens,vocab_size) 变为 (batch,vocab_size)
        logits = logits[:, -1, :]
        print(logits.shape)
        # 只关注最后一个时间步
        # 使用softmax函数计算概率
        # (batch,vocabsize)
        probas = torch.softmax(logits, dim=-1)
        # 归一化
        # 获取具有最高概率值的词汇索引
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        # 获取概率最高的词汇索引
        # 将采样的索引添加到序列中
        # (batch,n_tokens+1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [82]:
start_context = "Hello, I am"
# 模拟
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
# 进行语义理解
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [83]:
# 最终输出格式
# disable dropout
model.eval()
# 在检验的时候不需要正则化了
out = generate_text_simple(
    model=model,
    # 左边的参数名字,右边是函数传入的实际模型
    idx=encoded_tensor,  # 上下文的索引
    max_new_tokens=6,  # 最多运行六次,然后取结果概率最高的
    # 初始文本➕6
    context_size=GPT_CONFIG_124M["context_length"],
)

print("Output:", out)
print("Output length:", len(out[0]))
# 输出长度还有每个单词的id

tensor([[15496,    11,   314,   716]])
tensor([[[ 2.2484,  2.0364,  0.2140,  ..., -0.3425, -1.2684, -0.0087],
         [-4.3987, -0.5480,  0.2568,  ...,  1.6264, -1.0482, -1.9731],
         [-1.3833, -0.4516, -0.2062,  ..., -2.2153, -0.1814,  1.7350],
         [-2.4359, -0.7606,  0.3165,  ...,  0.2369, -3.3861,  0.8378]]])
tensor([[[ 2.4990,  1.8748,  0.0762,  ..., -0.5674, -1.1148, -0.2795],
         [-4.0539, -0.7496,  0.2197,  ...,  1.4834, -0.8403, -2.2330],
         [-1.1710, -0.5160, -0.2871,  ..., -2.4646,  0.0182,  1.4683],
         [-2.1633, -0.8835,  0.3412,  ...,  0.0313, -3.1466,  0.5158]]])
tensor([[[ 2.4867,  1.8998,  0.3285,  ..., -0.9222, -1.1350, -0.4625],
         [-4.0274, -0.9547, -0.2003,  ...,  1.4949, -0.5669, -2.1259],
         [-1.3260, -0.3017, -0.5144,  ..., -2.7653, -0.2854,  1.4535],
         [-2.1699, -0.8483, -0.0604,  ...,  0.0284, -3.2128,  0.1780]]])
tensor([[[ 2.3542,  1.8514,  0.7824,  ..., -0.6869, -0.8832, -0.5108],
         [-4.1161, -0.9065,  0.3

In [72]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I amAny answers when Dominion wrapped suff
